**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/as-with).**

---


# Introduction

You are getting to the point where you can own an analysis from beginning to end. So you'll do more data exploration in this exercise than you've done before.  Before you get started, run the following set-up code as usual. 

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex5 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-t6gwm4zn
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-t6gwm4zn
  Resolved https://github.com/Kaggle/learntools.git to commit 55574bf6512577688c9d0bcde665683903594910
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268951 sha256=50c7caf7dc531928e2227f96b37cbff3aef7c48e87d7f4737c20eda9bab91a30
  Stored in directory: /tmp/pip-ephem-wheel-cache-s9scfj2j/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


You'll work with a dataset about taxi trips in the city of Chicago. Run the cell below to fetch the `chicago_taxi_trips` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

You are curious how much slower traffic moves when traffic volume is high. This involves a few steps.

### 1) Find the data
Before you can access the data, you need to find the table name with the data.

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [7]:
[t.table_id for t in list(client.list_tables(dataset))]

['taxi_trips']

In [9]:
# Write the table name as a string below
table_name = "taxi_trips"

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [ ]:
#q_1.solution()

### 2) Peek at the data

Use the next code cell to peek at the top few rows of the data. Inspect the data and see if any issues with data quality are immediately obvious. 

In [12]:
table_ref = dataset_ref.table(table_name)
table = client.get_table(table_ref)
client.list_rows(table, max_results=10).to_dataframe()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,b3ec3cef49922433e4a5d8c74998af060224e308,e3dfdc6273b53e7bea4294c159fb810c907ffcd2aba9ba...,2015-01-24 21:00:00+00:00,2015-01-24 21:00:00+00:00,240,0.6,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,4.65,Cash,Dispatch Taxi Affiliation,41.907520,-87.626659,POINT (-87.6266589003 41.90752007470001),41.907492,-87.635760,POINT (-87.6357600901 41.9074919303)
1,7ded635e0822f2b6d854744d69caf601d8578fe3,19e804a8eab9224b352e6a384007418b519864a3e7c2d9...,2015-02-04 04:45:00+00:00,2015-02-04 05:00:00+00:00,540,2.9,NaN,NaN,22.0,24.0,...,3.0,12.25,Cash,Taxi Affiliation Services,41.922761,-87.699155,POINT (-87.69915534320002 41.9227606205),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001)
2,910e87798340a2b3e1739524f3b2ce9b3e6c62d5,1be31ca6ff375ee0a1a646ffeb90536c2a12453142c482...,2015-03-27 20:30:00+00:00,2015-03-27 20:45:00+00:00,600,1.7,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,7.65,Cash,Taxi Affiliation Services,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.909496,-87.630964,POINT (-87.630963601 41.9094956686)
3,89717b37316bb8a61c27335d56e4b22110b11e16,ea9c7f865233f880e5f00abb728092901eeaf52c85a8c1...,2015-01-29 16:00:00+00:00,2015-01-29 16:15:00+00:00,240,0.0,1.703132e+10,1.703108e+10,32.0,8.0,...,0.0,5.85,Cash,Choice Taxi Association,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259)
4,9c7429d65b397e01c79d8cc739cc3abaeac6ab84,0d61c4f9c8cb8d280fce388789bd7e26d2a692c0e510ca...,2014-12-13 15:30:00+00:00,2014-12-13 15:45:00+00:00,1200,2.7,1.703132e+10,1.703108e+10,32.0,8.0,...,1.0,11.45,Cash,Northwest Management LLC,41.877406,-87.621972,POINT (-87.6219716519 41.8774061234),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365)
5,57dc90cb57e771888096dd36fc9775630ecb1293,02e7e38cb3569f1f21634378f582ba3057396f1b9b3daa...,2015-03-27 19:15:00+00:00,2015-03-27 19:45:00+00:00,1740,0.3,NaN,NaN,8.0,23.0,...,0.0,21.55,Credit Card,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.900070,-87.720918,POINT (-87.7209182385 41.9000696026)
6,e4e82a5c9654eabf3e001c350d58873459533d81,773cd87ed1c8254ae93716d9edbaede8d97274688541e2...,2015-01-29 19:30:00+00:00,2015-01-29 19:30:00+00:00,240,0.9,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,6.45,Credit Card,Northwest Management LLC,41.905858,-87.630865,POINT (-87.6308650266 41.9058577688),41.898332,-87.620763,POINT (-87.6207628651 41.8983317935)
7,bfe2a8ce1032c0dae4e1e2416709c75f8bf4efe5,a6dda627be96fe26d3b2b06015a2a60358b81f91ca6c3a...,2015-01-29 19:30:00+00:00,2015-01-29 19:45:00+00:00,720,4.1,NaN,NaN,24.0,21.0,...,0.0,14.20,Credit Card,Choice Taxi Association,41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001),41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001)
8,a83b94613f700c44afb819294ead5910b168a27e,be69ef54513aa30c189bdce1568f720987e4bd02fe8734...,2015-02-06 00:30:00+00:00,2015-02-06 00:30:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,13.02,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
9,19dd343964962d968ec2062324fe17842f3bf16b,df6cb082a40e3306b18aad2e576da27b3b0cc855002ce9...,2015-01-25 02:45:00+00:00,2015-01-25 02:45:00+00:00,0,0.0,1.703183e+10,NaN,28.0,NaN,...,0.0,33.85,Cash,Taxi Affiliation Services,41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),NaN,NaN,None


After deciding whether you see any important issues, run the code cell below.

In [13]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
You can see the data by calling: 
```python
# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "taxi_trips" table
client.list_rows(table, max_results=5).to_dataframe()
```

Some location fields have values of `None` or `NaN`. That is a problem if we want to use those fields.


### 3) Determine when this data is from

If the data is sufficiently old, we might be careful before assuming the data is still relevant to traffic patterns today. Write a query that counts the number of trips in each year.  

Your results should have two columns:
- `year` - the year of the trips
- `num_trips` - the number of trips in that year

Hints:
- When using **GROUP BY** and **ORDER BY**, you should refer to the columns by the alias `year` that you set at the top of the **SELECT** query.
- The SQL code to **SELECT** the year from `trip_start_timestamp` is <code>SELECT EXTRACT(YEAR FROM trip_start_timestamp)</code>
- The **FROM** field can be a little tricky until you are used to it.  The format is:
    1. A backick (the symbol \`).
    2. The project name. In this case it is `bigquery-public-data`.
    3. A period.
    4. The dataset name. In this case, it is `chicago_taxi_trips`.
    5. A period.
    6. The table name. You used this as your answer in **1) Find the data**.
    7. A backtick (the symbol \`).

In [17]:
# Your code goes here
rides_per_year_query = """
    SELECT EXTRACT(YEAR FROM trip_start_timestamp) AS year, COUNT(1) AS num_trips
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    GROUP BY year
    ORDER BY year DESC
"""

# Set up the query (cancel the query if it would use too much of 
# your quota)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_year_query_job = client.query(rides_per_year_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_year_result = rides_per_year_query_job.to_dataframe()

# View results
print(rides_per_year_result)

# Check your answer
q_3.check()

    year  num_trips
0   2023    4337444
1   2022    6382071
2   2021    3947677
3   2020    3888831
4   2019   16476440
5   2018   20731105
6   2017   24979611
7   2016   31756403
8   2015   32385527
9   2014   37395079
10  2013   27217300


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_3.hint()
#q_3.solution()

### 4) Dive slightly deeper

You'd like to take a closer look at rides from 2016.  Copy the query you used above in `rides_per_year_query` into the cell below for `rides_per_month_query`.  Then modify it in two ways:
1. Use a **WHERE** clause to limit the query to data from 2016.
2. Modify the query to extract the month rather than the year.

In [26]:
rides_per_month_query = """
    SELECT
        EXTRACT(MONTH FROM trip_start_timestamp) AS month,
        COUNT(1) AS num_trips
    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE EXTRACT(YEAR FROM trip_start_timestamp) = 2016
    GROUP BY month
    ORDER BY month DESC
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
rides_per_month_query_job = client.query(rides_per_month_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
rides_per_month_result = rides_per_month_query_job.to_dataframe()

# View results
print(rides_per_month_result)

# Check your answer
q_4.check()

    month  num_trips
0      12    2312992
1      11    2387790
2      10    2725340
3       9    2532650
4       8    2629482
5       7    2682912
6       6    2841872
7       5    2859147
8       4    2854290
9       3    2851106
10      2    2568433
11      1    2510389


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [ ]:
#q_4.hint()
#q_4.solution()

### 5) Write the query

It's time to step up the sophistication of your queries.  Write a query that shows, for each hour of the day in the dataset, the corresponding number of trips and average speed.

Your results should have three columns:
- `hour_of_day` - sort by this column, which holds the result of extracting the hour from `trip_start_timestamp`.
- `num_trips` - the count of the total number of trips in each hour of the day (e.g. how many trips were started between 6AM and 7AM, independent of which day it occurred on).
- `avg_mph` - the average speed, measured in miles per hour, for trips that started in that hour of the day.  Average speed in miles per hour is calculated as `3600 * SUM(trip_miles) / SUM(trip_seconds)`. (The value 3600 is used to convert from seconds to hours.)

Restrict your query to data meeting the following criteria:
- a `trip_start_timestamp` > **2016-01-01** and < **2016-04-01**
- `trip_seconds` > 0 and `trip_miles` > 0

You will use a common table expression (CTE) to select just the relevant rides.  Because this dataset is very big, this CTE should select only the columns you'll need to create the final output (though you won't actually create those in the CTE -- instead you'll create those in the later **SELECT** statement below the CTE).

This is a much harder query than anything you've written so far.  Good luck!

In [63]:
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT
                       EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day,
                       trip_miles,
                       trip_seconds
                   FROM
                       `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE
                       trip_start_timestamp > '2016-01-01' AND
                       trip_start_timestamp < '2016-04-01' AND
                       trip_seconds > 0 AND
                       trip_miles > 0
               )
               SELECT
                   hour_of_day,
                   COUNT(1) AS num_trips,
                   3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
               FROM
                   RelevantRides
               GROUP
                   BY hour_of_day
               ORDER
                   BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe()

# View results
print(speeds_result)

# Check your answer
q_5.check()

    hour_of_day  num_trips    avg_mph
0             0     203092  20.191744
1             1     178046  18.628598
2             2     143447  18.444370
3             3     108899  19.273107
4             4      80067  27.599669
5             5      75786  33.065604
6             6     102254  28.533112
7             7     187585  19.884592
8             8     284223  16.787900
9             9     306854  18.434124
10           10     279762  20.091309
11           11     294006  20.926340
12           12     311522  20.063901
13           13     317225  19.766321
14           14     312629  19.309655
15           15     319953  18.515564
16           16     349455  17.168814
17           17     394324  14.641375
18           18     431991  15.381995
19           19     416743  17.795008
20           20     356279  20.347398
21           21     318363  22.584731
22           22     289886  21.129847
23           23     241690  20.259757


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

That's a hard query. If you made good progress towards the solution, congratulations!

# Keep going

You can write very complex queries now with a single data source. But nothing expands the horizons of SQL as much as the ability to combine or **JOIN** tables.

**[Click here](https://www.kaggle.com/dansbecker/joining-data)** to start the last lesson in the Intro to SQL course.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*